**ML PROBLEM 5.1**

In [42]:
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.linear_model import LassoCV, Lasso
from sklearn import linear_model 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from math import *
import warnings
import os
from sklearn import preprocessing
from sklearn.model_selection import cross_validate
from sklearn.metrics import r2_score
warnings.filterwarnings("ignore")

In [3]:
X_train = np.load(os.path.abspath('Xtrain_Classification1.npy'))
y_train = np.load(os.path.abspath('ytrain_Classification1.npy'))
X_test  = np.load(os.path.abspath('Xtest_Classification1.npy'))

In [4]:
print(X_train.shape)
# 6254 elements with 2352 attributes
print(X_test.shape)
print(y_train.shape)


#Checking the imbalance in the data
y_melanoma=[y for y in y_train if y]
y_nevu=[y for y in y_train if not y]

print(len(y_melanoma))
print(len(y_nevu))

(6254, 2352)
(1764, 2352)
(6254,)
896
5358


prof da cadeira diz pa usar transfer learning


podemos só usar classificadores do sklearn tipo mlpclassifier, logistic regression, naive bayes, svm

balanced accuracy: (sensitivity + setificity)/2
sensitivity= True positive/(true positvity + false negativity)
setificity = (true negative)/(true neg + false positive)


Use pytorch neural network - harder but most usefull for harder problems (other options are jax and keras)
output layer - 2 neurons is a classification problem - use softmax (if it was 1 neuron it would be regression)



In [ ]:
#balanced accuracy
def balanced_accuracy(y, y_predicted):
    TN=0
    TP=0
    FP=0
    FN=0
    for i in range(y_predicted.shape[0]):
        if y_predicted(i):
            if y_predicted(i)==y(i):
                TP+=1
            else:
                FP+=1
        else:
            if y_predicted(i)==y(i):
                TN+=1
            else:
                FN+=1
    sensitivity= TP/(TP+FN)
    specificity = TN/(TN/FP)
    balanced_accuracy_val=   (sensitivity+specificity)/2
    return balanced_accuracy_val


**Neural Network**


In [4]:
#import tensorflow as tf
import keras
from keras import *

In [5]:
def balanced_accuracy(y_pred, y_true):
    neg_y_true = 1 - y_true
    neg_y_pred = 1 - y_pred

    fp = keras.sum(neg_y_true * y_pred)
    tn = keras.sum(neg_y_true * neg_y_pred)
    fn = keras.sum(neg_y_pred*y_true)
    tp = keras.sum(y_true*y_pred)

    specificity = tn / (tn + fp)
    sensitivity = tp / (tp + fn)
    balanced_auc = (specificity + sensitivity)/2

    return balanced_auc

In [6]:
def balanced_accuracy2(y_pred, y_true):
    neg_y_true = 1 - y_true
    neg_y_pred = 1 - y_pred

    fp = keras.sum(neg_y_true * y_pred)
    tn = keras.sum(neg_y_true * neg_y_pred)
    fn = keras.sum(neg_y_pred*y_true)
    tp = keras.sum(y_true*y_pred)

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    balanced_auc2 = (precision + recall)/2

    return balanced_auc2

In [163]:
### Neural Network
model_keras = keras.Sequential()
model_keras.add(layers.Dense(588, activation="relu", name="Input"))
model_keras.add(layers.Dense(147, activation="relu", name="layer2"))
model_keras.add(layers.Dense(1, activation="sigmoid", name="Output"))

model_keras.compile(
    optimizer=keras.optimizers.Adam(),  # Optimizer: use either ADAM or SGDM
    # Loss function to minimize
    loss=keras.losses.BinaryCrossentropy(),
    # List of metrics to monitor
    metrics=[keras.metrics.Precision(),keras.metrics.Recall()]
)

In [164]:
from sklearn.preprocessing import StandardScaler

X_val, X_train_sample = X_train[:500].copy(), X_train[500:].copy()
y_val, y_train_sample = y_train[:500].copy(), y_train[500:].copy()

# Create an instance of the scaler
scaler = StandardScaler()

# Fit on training data
scaler.fit(X_train)

# Transform both training and test data
X_train_sample = scaler.transform(X_train_sample)
X_val = scaler.transform(X_val)

model_trained = model_keras.fit(
    X_train_sample,
    y_train_sample,
    batch_size=64,
    epochs=20,
    # We pass some validation for
    # monitoring validation loss and metrics
    # at the end of each epoch
    validation_data=(X_val, y_val),
)

Epoch 1/20
90/90 [==============================] - 9s 31ms/step - loss: 0.5071 - precision_27: 0.3589 - recall_27: 0.2625 - val_loss: 0.3782 - val_precision_27: 0.5294 - val_recall_27: 0.1169
Epoch 2/20
90/90 [==============================] - 2s 26ms/step - loss: 0.3271 - precision_27: 0.5242 - recall_27: 0.3040 - val_loss: 0.3280 - val_precision_27: 0.5435 - val_recall_27: 0.3247
Epoch 3/20
90/90 [==============================] - 2s 26ms/step - loss: 0.2938 - precision_27: 0.5693 - recall_27: 0.3211 - val_loss: 0.3176 - val_precision_27: 0.6500 - val_recall_27: 0.1688
Epoch 4/20
90/90 [==============================] - 2s 27ms/step - loss: 0.2755 - precision_27: 0.6738 - recall_27: 0.3858 - val_loss: 0.3074 - val_precision_27: 0.7500 - val_recall_27: 0.2727
Epoch 5/20
90/90 [==============================] - 3s 28ms/step - loss: 0.2621 - precision_27: 0.6460 - recall_27: 0.3944 - val_loss: 0.3373 - val_precision_27: 0.5349 - val_recall_27: 0.2987
Epoch 6/20
90/90 [=================

In [165]:
dtf = pd.DataFrame(model_trained.history)
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(9,9))
fig.suptitle('Model Evaluation')

ax1.plot((dtf.iloc[:,4]+dtf.iloc[:,5])/2,'go--',label='Balanced Accuracy')
ax1.legend()
ax1.set_xlabel('Epochs')

ax2.plot(dtf.iloc[:,4],'bo--',label='val precision')
ax2.plot(dtf.iloc[:,5],'ro--',label='val recall')
ax2.legend()
ax2.set_xlabel('Epochs')

plt.show()

Network using image information


In [7]:

novo = np.reshape(X_train,(6254,28,28,3))
print(novo)

[[[[199 172 189]
   [196 169 186]
   [196 167 185]
   ...
   [196 172 185]
   [195 172 182]
   [193 170 178]]

  [[195 168 185]
   [196 169 186]
   [199 170 188]
   ...
   [198 174 187]
   [196 173 183]
   [195 172 180]]

  [[194 170 186]
   [197 173 189]
   [203 176 193]
   ...
   [199 175 188]
   [198 175 185]
   [197 174 184]]

  ...

  [[186 160 161]
   [188 163 166]
   [192 167 171]
   ...
   [176 148 134]
   [184 154 143]
   [186 156 145]]

  [[178 153 156]
   [179 157 159]
   [183 161 164]
   ...
   [188 158 148]
   [187 154 147]
   [184 151 144]]

  [[171 149 151]
   [175 153 155]
   [179 157 160]
   ...
   [187 157 149]
   [182 149 144]
   [178 145 140]]]


 [[[241 184 199]
   [240 186 200]
   [241 187 203]
   ...
   [235 175 187]
   [230 170 182]
   [227 167 179]]

  [[242 185 200]
   [241 184 199]
   [238 184 200]
   ...
   [237 177 189]
   [232 172 184]
   [230 170 182]]

  [[242 183 201]
   [240 183 200]
   [236 182 198]
   ...
   [238 180 194]
   [233 175 187]
   [231 173

Epoch 1/3
45/45 [==============================] - 3s 37ms/step - loss: 0.8639 - precision_27: 0.1448 - recall_27: 0.0781 - val_loss: 0.3809 - val_precision_27: 0.0000e+00 - val_recall_27: 0.0000e+00
Epoch 2/3
45/45 [==============================] - 1s 32ms/step - loss: 0.3548 - precision_27: 0.3478 - recall_27: 0.0098 - val_loss: 0.3591 - val_precision_27: 0.7000 - val_recall_27: 0.0909
Epoch 3/3
45/45 [==============================] - 1s 31ms/step - loss: 0.3360 - precision_27: 0.4865 - recall_27: 0.0220 - val_loss: 0.3427 - val_precision_27: 0.5000 - val_recall_27: 0.0130


In [117]:
model2, model_history2 = neural_defandTrain("adam", "binary_crossentropy" ,3,128, X_train, y_train)


In [115]:

def neural_defandTrain(optimizer_string, loss_string ,epoch_nr,batch, x_all, y_all):

    X_train_sample, y_train_sample, X_validation, y_validation = x_y_preprocessing (x_all, y_all)

    model = model_def(optimizer_string, loss_string)

    model_history = model.fit(
        X_train_sample,
        y_train_sample,
        batch_size=batch,
        epochs=epoch_nr,
        # We pass some validation for
        # monitoring validation loss and metrics
        # at the end of each epoch
        validation_data=(X_validation, y_validation)
    )
    best_balancedAccuracy = plot_neuralmodel(model_history)
    
    save_model(model, model_history, optimizer_string, loss_string ,epoch_nr,batch, best_balancedAccuracy)

    return model, model_history


In [58]:
### Neural Network
def model_def(optimizer_string, loss_string ):
    model_img= keras.Sequential()
    model_img.add(layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28,28,3)))
    model_img.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model_img.add(layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model_img.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model_img.add(layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model_img.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model_img.add(layers.Flatten())
    model_img.add(layers.Dense(256, activation='relu'))
    model_img.add(layers.Dense(64, activation='relu'))
    model_img.add(layers.Dense(64, activation='relu'))
    model_img.add(layers.Dense(1, activation='sigmoid'))
    #model_img.add(layers.Dense(2, activation='softmax'))
    
    model_img.compile(
            
            
            optimizer = optimizer_string,
            #optimizer=keras.optimizers.Adam(),  # Optimizer: use either ADAM or SGDM
            # Loss function to minimize
            #loss=keras.losses.BinaryCrossentropy(),
            loss = loss_string,
            # List of metrics to monitor
            metrics=[keras.metrics.Precision(),keras.metrics.Recall()]
        )

    return model_img



In [116]:
from pathlib import Path, PureWindowsPath

def save_model (model_fitted, model_history, optimizer_string, loss_string ,epoch_nr,batch,best_balancedAccuracy ):
    
    #define names for the files
    file_name_model=f"bestbacc_{best_balancedAccuracy}  {optimizer_string}  {loss_string}  Epoch_{epoch_nr}  Batch_{batch}  Model.keras"
    #model_path=Path(r'C:\Users\alexa\OneDrive - Universidade de Lisboa\4º Ano\1º Semestre - MEEC\Aprendizagem Automática\Labs2\ML_Aaut',file_name_model)
    file_name_history=f"bestbacc_{best_balancedAccuracy}  {optimizer_string}  {loss_string}  Epoch_{epoch_nr}  Batch_{batch}  History.npy"
    #history_path=Path(r'C:\Users\alexa\OneDrive - Universidade de Lisboa\4º Ano\1º Semestre - MEEC\Aprendizagem Automática\Labs2\ML_Aaut',file_name_history)

    #save the model in a .keras file
    model_fitted.save(file_name_model)
    #save history to numpy array
    np.save(file_name_history,model_history.history)







    

In [114]:
def plot_neuralmodel(model):
    dtf = pd.DataFrame(model.history)
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(9,9))
    fig.suptitle('Model Evaluation')

    best_bacc =round(max((dtf.iloc[:,4]+dtf.iloc[:,5])/2),4)
   

    ax1.plot((dtf.iloc[:,4]+dtf.iloc[:,5])/2,'go--',label=f'Balanced Accuracy\nBest balanced accuracy {best_bacc}')
    
    ax1.legend()
    ax1.set_xlabel('Epochs')

    ax2.plot(dtf.iloc[:,4],'bo--',label='val precision')
    ax2.plot(dtf.iloc[:,5],'ro--',label='val recall')
    ax2.legend()
    ax2.set_xlabel('Epochs')

    plt.show()
    return best_bacc






In [56]:
def x_y_preprocessing(x,y):
    X_train_img = np.reshape(X_train,(6254,28,28,3)).copy()
    X_val, X_train_sample = X_train_img[:500].copy(), X_train_img[500:].copy()
    y_val, y_train_sample = y_train[:500].copy(), y_train[500:].copy()

    y_train_2labels=np.empty([0,2]) #first column is to say if it is nervu y=0 and second column is to say if it is melanoma y=1
    for i in y_train_sample:
        if i: #melanoma
            y_train_2labels=np.vstack((y_train_2labels, np.array([1,0])))
        else: #nervu
            y_train_2labels=np.vstack((y_train_2labels, np.array([0,1])))
    return X_train_sample, y_train_sample, X_val, y_val

